In [1]:
import pandas as pd
import duckdb
import plotly.express as px
from pycaret.classification import *
import os
import io
import numpy as np

In [2]:
import re
from pathlib import Path
from typing import TypedDict


class AmbulanceEvent(TypedDict):
    seq: str
    customer: str
    sse: str
    ambnum: str
    payload: str


def get_event(path: Path, *, prefix: str) -> list[AmbulanceEvent]:
    chunks: list[AmbulanceEvent] = []
    with open(path, "r", encoding="cp1250") as file:
        buffer: str = ""
        last_match: list[str] = []

        for line in file:
            if line.strip().startswith("===") and line.count("/") == 2:
                match = re.match(
                    r"=== (\d+)\. (\d+) / ([a-zA-Z0-9]{10}) / (\d+)", line.strip()
                )
                if last_match:
                    _, serial, sse, ambnum = last_match
                    chunks.append(
                        {
                            "FILE": os.path.splitext(path.name)[0].split(prefix)[1],
                            "SERIAL": int(serial),
                            "RC": sse,
                            "AMBNUM": int(ambnum),
                            "CONTENT": buffer,
                        }
                    )

                buffer = ""
                last_match = match.groups()
                continue

            buffer += line
    return chunks


zpravy = []
for file in sorted((Path("../../data/version2/PAC/AMBULATNÍ ZPRÁVA/2023")).glob("*.Txt")):
    zpravy.extend(get_event(file, prefix="Events_"))

zpravy = pd.DataFrame(zpravy)
display(zpravy.head())

propustky = []
for file in sorted((Path("../../data/version2/PAC/PROPOUŠTĚCÍ ZPRÁVA/2023")).glob("*.Txt")):
    propustky.extend(get_event(file, prefix="ZavZprav_"))
    
propustky = pd.DataFrame(propustky)
display(propustky.head())


,FILE,SERIAL,RC,AMBNUM,CONTENT
0,2023_01,1430073,640XXXXXXX,53835,MUDr. Pajdlhauser Roman\ndomluva na hosital CH...
1,2023_01,1430084,716XXXXXXX,58891,MUDr. Hanušová Zuzana\nC19 \nPravidelná kontro...
2,2023_01,1387088,660XXXXXXX,55647,MUDr. Urie Petr\nC051 \nVýška: 170cm Váha: ...
3,2023_01,1423870,685XXXXXXX,26033,MUDr. Nohejlová Medková Anna\nC493 \nC810 \n3...
4,2023_01,1399503,660XXXXXXX,50413,MUDr. Urie Petr\nC162 \nVýška: 165cm Váha:...


,FILE,SERIAL,RC,AMBNUM,CONTENT
0,2023_01,275828,665XXXXXXX,59047,DŮVOD PŘIJETÍ K HOSPITALIZACI: \nPacientku při...
1,2023_01,275913,835XXXXXXX,50775,DŮVOD PŘIJETÍ K HOSPITALIZACI: \npřijata k apl...
2,2023_01,276472,560XXXXXXX,8460,DŮVOD PŘIJETÍ K HOSPITALIZACI: \npřijat k apli...
3,2023_01,276172,990XXXXXXX,59522,DŮVOD PŘIJETÍ K HOSPITALIZACI: \nzpomalené psy...
4,2023_01,277031,560XXXXXXX,54409,DŮVOD PŘIJETÍ K HOSPITALIZACI: \nPacient přija...


In [3]:
vazby = pd.read_csv("../../data/version2/PAC/events_date_2023.csv", encoding="windows-1252", sep=";")

In [4]:
vazby.head()

,SERIAL,AMBNUM,DATFR,DEPARTM,WHO
0,1387088,55647,16.01.2023 11:00,10370,22298
1,1399503,50413,05.01.2023 11:00,10370,22298
2,1399525,56795,12.01.2023 09:45,10370,22298
3,1407661,14010,17.01.2023 06:42,10370,22161
4,1408001,23454,26.01.2023 09:12,10452,22161


In [5]:
def read_txtevent_file(path: Path, *, with_header: bool = False) -> str:
    with open(path, "r", encoding="cp1250") as file:
        file_prefix = path.name.split(".")[0].split("TXTEVENT_")[1]
        return "".join(
            [
                (file_prefix if enum > 0 else "FILE") + ";" + line
                for enum, line in enumerate(file)
                if with_header or enum > 0
            ]
        )


lines = ""
for i, map_file in enumerate(sorted((Path("../../data/version2/PAC/DOKUMENTACE/2023")).glob("*.csv"))):
    lines += read_txtevent_file(map_file, with_header=i == 0)

dokumentace = pd.read_csv(io.StringIO(lines), sep=";")
dokumentace.head()

,FILE,POR,UDALOST_CISLO,UDALOST,DATUM,ODDEL,LEKAR,DATUM_CAS,DIAGNOZA,DATUM_OD,...,HODNOCENI,JMENO,RC,AMBNUM,CISPAC,PSC,DATUM_NAR,VEKR,VEKM,VEKD
0,2023_01,1,2,RATO,20230125,10248,22217,20230125_101800,NaN,00010101_000000,...,0,NaN,640XXXXXXX,53835,2124171,11000,10101,58,8,14
1,2023_01,2,5,RATP,20230125,10370,93786,20230125_102500,C19,00010101_000000,...,0,NaN,716XXXXXXX,58891,2259382,28601,10101,51,3,14
2,2023_01,3,5,RATP,20230116,10370,22298,20230116_110000,C051,00010101_000000,...,0,NaN,660XXXXXXX,55647,2193508,27201,10101,56,9,29
3,2023_01,4,5,RATP,20230104,10452,22422,20230104_112500,"C493, C810",00010101_000000,...,0,NaN,685XXXXXXX,26033,508766,16000,10101,54,6,19
4,2023_01,5,5,RATP,20230105,10370,22298,20230105_110000,C162,00010101_000000,...,0,NaN,660XXXXXXX,50413,298937,10000,10101,56,10,13


In [6]:
ucty = pd.read_csv("../../data/version2/VYK_23/vyk_23_vykpac_new.csv", encoding="windows-1252", sep=";")

In [7]:
ucty.head()

,CDOKL,POJCDOKL,CYEAR,CCHOR1,CIDENT,CISPAC,POJISTOVNA,CISLOPOJISTKY,PRIPOJISTENI,POHLAVI,...,OBDOBIDOKL,PLATCE,TYPSUBJEKTU,KODSUBJEKTU,SYSTEMUCETZALOZIL,SOURCESYSTEM,CISPOJUZIS,UNISFAKT1,DATODSOUHLUCET,KDOODSOUHLUCET
0,6384,6384,2023,45968574,7290224,213292,207,2230734.0,0,0,...,20230101,0,5,10626,3,3,NaN,0,17.01.2023 18:37,NaN
1,6386,6386,2023,45980138,7290261,213292,207,2230734.0,0,0,...,20230101,0,5,10626,3,3,NaN,0,18.01.2023 15:08,NaN
2,6442,6442,2023,45964330,NaN,1892691,201,NaN,0,0,...,20230101,0,5,10837,219,219,NaN,0,20.01.2023 09:51,NaN
3,6630,6630,2023,46044195,NaN,209185,207,NaN,0,0,...,20230101,0,5,10370,3,3,NaN,0,26.01.2023 11:38,NaN
4,6790,6790,2023,45907288,7290108,1640608,209,8082470.0,0,0,...,20230101,0,5,10626,3,3,NaN,0,09.01.2023 23:50,NaN


In [8]:
materialy = pd.read_csv("../../data/version2/VYK_23/vyk_23_material_new.csv", encoding="windows-1252", sep=";")

In [9]:
materialy.head()

,SERIAL,CDOKL,DATUM,TYP,KOD,ZVL,LOKALIZACE,ODDELENI,MNOZSTVI,CENAUZIV,CENA,SERIALCDB,CISPAC
0,262,12590,25.01.2023 00:00,1,0000516,0,NaN,10626,0.1,0.000,0.240,262,1570034
1,263,12590,25.01.2023 00:00,1,0093746,0,NaN,10626,0.2,0.000,23.460,263,1570034
2,264,12590,25.01.2023 00:00,3,0048092,0,NaN,10626,1.0,0.000,154.000,264,1570034
3,349,14330,07.01.2023 00:00,1,0000516,0,NaN,10626,0.1,0.000,0.240,349,2104772
4,350,14330,07.01.2023 00:00,1,0093746,0,NaN,10626,0.1,0.000,11.730,350,2104772


In [10]:
vykony = pd.read_csv("../../data/version2/VYK_23/vyk_23_vykony_new.csv", encoding="windows-1252", sep=";")

In [11]:
vykony.head()

,SERIAL,CDOKL,DATUM,KOD,ODBORNOST,DG,LOKALIZACE,ODDELENI,CAS,MNOZSTVI,BODY,CENAMAT,CENAUZIV,CENA,SERIALCDB,CISPAC
0,6198,6384,17.01.2023 00:00,42022,402.0,C719,NaN,10626,NaN,1,435.000,0.0,0.0,391.500,6198.0,213292
1,6199,6384,17.01.2023 00:00,9543,402.0,C719,NaN,10626,NaN,1,0.000,0.0,0.0,0.000,6199.0,213292
2,6200,6386,18.01.2023 00:00,9511,402.0,C719,NaN,10626,NaN,1,45.000,0.0,0.0,40.500,6200.0,213292
3,6317,6442,17.01.2023 00:00,42022,402.0,N47,NaN,10837,NaN,1,435.000,0.0,0.0,391.500,6317.0,1892691
4,6318,6442,17.01.2023 00:00,9543,402.0,N47,NaN,10837,NaN,1,0.000,0.0,0.0,0.000,6318.0,1892691


In [12]:
res = duckdb.sql(
    """
    with zpravy_a_vazby as (
        select
            z.serial as zprava_serial,
            z.rc,
            z.ambnum,
            z.content,
            date_trunc('day', STRFTIME(STRPTIME(vazby.DATFR, '%d.%m.%Y %H:%M'), '%Y-%m-%dT%H:%M:%S')::timestamp) as datum_a_cas_zpravy,
            vazby.DEPARTM,
            vazby.WHO,
        from zpravy z
            left join vazby using (serial, ambnum)
    )
    , zpravy_a_vazby_a_dokumentace_tmp as (
        select distinct  -- why do we have duplicates
            datum_a_cas_zpravy,
            date_trunc('day', strptime(d.DATUM_CAS::text, '%Y%m%d_%H%M%S')) as datum_a_cas_dokumentace,
            zv.ambnum,
            zv.content,
            d.CISPAC,
        from zpravy_a_vazby zv
            left join dokumentace d
                on zv.ambnum = d.ambnum
                and date_trunc('day', strptime(d.DATUM_CAS::text, '%Y%m%d_%H%M%S')) >= datum_a_cas_zpravy
    )
    , zpravy_a_vazby_a_dokumentace as (
        select
            datum_a_cas_zpravy,
            ambnum,
            array_agg(distinct content) as contents,
            min(cispac) as cispac,
        from zpravy_a_vazby_a_dokumentace_tmp
        group by 
            datum_a_cas_zpravy,
            ambnum
    )
    , zpravy_a_vazby_a_dokumentace_a_vykony_tmp as (
        select
            date_trunc('day', strptime(datum::text, '%d.%m.%Y %H:%M')) as datum_vykonu,
            z.*,
            u.CDOKL,
            u.kod as kod_vykonu,
            u.odbornost,
            u.mnozstvi as mnozstvi_vykonu,
            u.body,
        from zpravy_a_vazby_a_dokumentace z
            left join vykony u on
                z.cispac = u.cispac
                and date_trunc('day', strptime(datum::text, '%d.%m.%Y %H:%M')) = datum_a_cas_zpravy
    )
    , zpravy_a_vazby_a_dokumentace_a_vykony as (
        select
            datum_a_cas_zpravy,
            contents,
            AMBNUM,
            cispac,
            array_agg(
                struct_pack(
                    CDOKL,
                    kod_vykonu,
                    odbornost,
                    mnozstvi_vykonu,
                    body
                )
            ) as vykony
        from zpravy_a_vazby_a_dokumentace_a_vykony_tmp
        group by 
            datum_a_cas_zpravy,
            contents,
            AMBNUM,
            cispac,
    )
    , zpravy_a_vazby_a_dokumentace_a_vykony_a_materialy_tmp as (
        select
            z.*,
            m.cdokl,
            m.kod as kod_materialu,
            m.mnozstvi as mnozstvi_materialu,
        from zpravy_a_vazby_a_dokumentace_a_vykony z
            left join materialy m on
                z.cispac = m.cispac
                and date_trunc('day', strptime(m.datum::text, '%d.%m.%Y %H:%M')) = datum_a_cas_zpravy
    )
    , zpravy_a_vazby_a_dokumentace_a_vykony_a_materialy as (
        select
            datum_a_cas_zpravy,
            contents,
            AMBNUM,
            cispac,
            vykony,
            array_agg(
                struct_pack(
                    cdokl,
                    kod_materialu,
                    mnozstvi_materialu
                )
            ) as materialy
        from zpravy_a_vazby_a_dokumentace_a_vykony_a_materialy_tmp
        group by 
            datum_a_cas_zpravy,
            contents,
            AMBNUM,
            cispac,
            vykony,
    )
    select *
    from zpravy_a_vazby_a_dokumentace_a_vykony_a_materialy
    """
).df()

In [13]:
vykony_pivot = pd.get_dummies(vykony.set_index('CDOKL')['KOD']).groupby('CDOKL').sum()

In [14]:
vykony_pivot

,41,42,6134,9111,9113,9115,9117,9119,9125,9127,...,99878,99879,99880,99881,99882,99883,99893,99895,99976,99991
CDOKL,,,,,,,,,,,,,,,,,,,,,
6384,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6386,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6442,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6630,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6790,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5615486,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5615508,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5615514,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
co_occurrence_matrix = np.dot(vykony_pivot.T, vykony_pivot)

In [16]:
co_occurrence_matrix

array([[  18,    0,    3, ...,    0,    5,    0],
       [   0,    3,    0, ...,    0,    0,    0],
       [   3,    0,  186, ...,    0,   21,    0],
       ...,
       [   0,    0,    0, ...,    8,    0,    0],
       [   5,    0,   21, ...,    0, 5226,    1],
       [   0,    0,    0, ...,    0,    1, 1481]])

In [17]:
np.fill_diagonal(co_occurrence_matrix, 0)

In [18]:
co_occurrence_df = pd.DataFrame(co_occurrence_matrix, index=vykony_pivot.columns, columns=vykony_pivot.columns)
display(co_occurrence_df)

,41,42,6134,9111,9113,9115,9117,9119,9125,9127,...,99878,99879,99880,99881,99882,99883,99893,99895,99976,99991
41,0,0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,0
42,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6134,3,0,0,0,0,0,0,0,0,0,...,0,7,2,1,0,0,0,0,21,0
9111,0,0,0,0,0,13,1,3,0,0,...,0,0,0,0,0,0,0,0,0,0
9113,0,0,0,0,0,1,0,4,13,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99883,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
99893,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99895,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
99976,5,0,21,0,0,0,0,968,0,8,...,0,2,3,10,0,0,0,0,0,1


In [19]:
co_occurrence_df.reset_index()

,index,41,42,6134,9111,9113,9115,9117,9119,9125,...,99878,99879,99880,99881,99882,99883,99893,99895,99976,99991
0,41,0,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,0
1,42,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,6134,3,0,0,0,0,0,0,0,0,...,0,7,2,1,0,0,0,0,21,0
3,9111,0,0,0,0,0,13,1,3,0,...,0,0,0,0,0,0,0,0,0,0
4,9113,0,0,0,0,0,1,0,4,13,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,99883,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
83,99893,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
84,99895,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
85,99976,5,0,21,0,0,0,0,968,0,...,0,2,3,10,0,0,0,0,0,1


In [20]:
# Convert to DataFrame for easier visualization

fig = px.imshow(co_occurrence_df, 
                labels=dict(x="Code 1", y="Code 2", color="Co-occurrence Count"),
                title="Code Co-occurrence Matrix",
                color_continuous_scale="YlGnBu")

# Show the plot
fig.show()


In [21]:
co_occurrence_with_index = co_occurrence_df.reset_index()

In [22]:
co_occurrence_df["max_occurence"] = co_occurrence_df.max(axis=1)
co_occurrence_df['max_column'] = co_occurrence_df.idxmax(axis=1)

In [23]:
co_occurrence_df.sort_values(by="max_occurence")[["max_occurence", "max_column"]]

,max_occurence,max_column
22365,0,41
9551,0,41
51811,1,9543
25135,1,9119
42,2,9227
...,...,...
9561,5666,9543
9223,8369,42520
42520,9094,9543
9543,31721,42022


In [ ]:
vykony_pivot = pd.get_dummies(vykony.set_index('CDOKL')['KOD']).groupby('CDOKL').sum()
co_occurrence_matrix = np.dot(vykony_pivot.T, vykony_pivot)
np.fill_diagonal(co_occurrence_matrix, 0)
co_occurrence_df = pd.DataFrame(co_occurrence_matrix, index=vykony_pivot.columns, columns=vykony_pivot.columns)
co_occurrence_df.reset_index(inplace=True)
co_occurrence_df_final = co_occurrence_df.rename(columns={'index': 'kod'})

In [29]:
duckdb.sql("""
    select
        kod,
        count(distinct cdokl) cnt
    from vykony
    group by kod
    order by cnt desc
    limit 10
""")

┌───────┬───────┐
│  KOD  │  cnt  │
│ int64 │ int64 │
├───────┼───────┤
│  9543 │ 37928 │
│ 42022 │ 37403 │
│  9119 │ 14018 │
│  9511 │ 13756 │
│ 42520 │ 12055 │
│  9223 │  8685 │
│  9561 │  7429 │
│  9541 │  7146 │
│  9215 │  5657 │
│ 42023 │  5385 │
├───────┴───────┤
│    10 rows    │
└───────────────┘